# Import bibilotek

In [30]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error


In [2]:
pip install --upgrade tables

Note: you may need to restart the kernel to use updated packages.


# Odczyt danych z pliku h5

In [10]:
df_train = pd.read_hdf("train_data.h5")
df_train['price'] = df_train['price'].map(parse_price)

df_test = pd.read_hdf("test_data.h5")

df = pd.concat([df_train, df_test])
print(df_train.shape, df_test.shape)

df

(22732, 8) (11448, 7)


,geo_block,breadcrumbs,price,owner,params,date,user_block,id
0,"[г. Москва, Лианозово, г. Москва, Лианозово]","[Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]",5.402084,[],"{'Охрана:': 'закрытая территория', 'Тип здания...","[22 апреля, 12, (+1 за сегодня), Обновлено 18 ...","[<div class=""block-user__name"">НДВ Недвижимост...",34995
1,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",6.827990,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 2, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",15030
2,"[г. Москва, ул Лобачевского, г. Москва, ул Лоб...","[Москва, МЦК Кутузовская, ул. Лобачевского]",8.680000,"[ЖК ""Крылья"". Доп. скидка 5%!]","{'Лифт:': 'да', 'Охрана:': 'закрытая территори...","[20 марта, 12, (+2 за сегодня), Обновлено 18 мая]",[],37811
3,"[г. Москва, Ховрино, ул Дыбенко, г. Москва, Хо...","[Москва, Ховрино, м. Ховрино, МЦК Коптево, ул....",8.624435,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[29 апреля, 3, (+1 за сегодня), Обновлено 13 мая]","[<div class=""block-user__name"">MR-Group</div>,...",31685
4,"[г. Москва, Даниловский, г. Москва, Даниловский]","[Москва, Даниловский, м. Автозаводская, МЦК Ав...",29.500000,[],"{'Общая площадь:': '141.1 м²', 'Дата публикаци...","[15 декабря 2018, 30, (+1 за сегодня), Обновле...","[<div class=""block-user__name"">Наталья Вдовиче...",56215
...,...,...,...,...,...,...,...,...
11443,"[г. Москва, г. Москва]","[Москва, МЦК Кутузовская]",NaN,[],"{'Лифт:': 'да', 'Охрана:': 'предусмотрена', 'Т...","[17 мая, 3, (+2 за сегодня)]","[<div class=""block-user__name"">MR Group</div>,...",47294
11444,"[г. Москва, Хорошёво-Мнёвники, г. Москва, Хоро...","[Москва, Хорошёво-Мнёвники, м. Октябрьское пол...",NaN,[],"{'Лифт:': 'да', 'Охрана:': 'предусмотрена', 'Т...","[16 мая, 1, (+1 за сегодня)]","[<div class=""block-user__name"">Офис продаж ЖК ...",14091
11445,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",NaN,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 1, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",13059
11446,"[Новая Москва, п. Внуковское, г. Москва, Новая...","[Москва, Новая Москва, п. Внуковское, МЦК Куту...",NaN,[Переделкино Ближнее],"{'Лифт:': 'да', 'Охрана:': 'консьерж', 'Тип зд...","[вчера в 10:14, 2, (+2 за сегодня)]",[],10144


# Funkcje pomocniczne

In [11]:
def parse_price(val):
    if isinstance(val, str): 
        if "₽" in val:
            val = val.split('₽')[0]
            
        val = val.replace(' ', '')
        return int(val) / 1000000
    
    return float(val)

def parse_area(val):
    if isinstance(val, int): return val
    if isinstance(val, float): return val
    
    return float(val.split("м")[0].replace(" ", ""))

def parse_floor(val):
    if isinstance(val, int): return val
    if isinstance(val, str):
        return val.split('/')[0]
    return val

def get_metro_station(row):
        for i in row:
            if 'МЦК' in i:
                return i
            
def check_log_model(df, feats, model, cv=5, scoring="neg_mean_absolute_error"):
    df_train = df[ ~df["price"].isnull() ].copy()

    X = df_train[feats]
    y = df_train["price"]
    y_log = np.log(y)
    
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    scores = []
    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_log_train, y_test = y_log.iloc[train_idx], y.iloc[test_idx]

        model = xgb.XGBRegressor(max_depth=md, n_estimators=ne, learning_rate=lr, random_state=0)
        model.fit(X_train, y_log_train)
        y_log_pred = model.predict(X_test)
        y_pred = np.exp(y_log_pred)

        score = mean_absolute_error(y_test, y_pred)
        scores.append(score)

    return np.mean(scores), np.std(scores)


# Future ennginiring

In [12]:
params = df["params"].apply(pd.Series)

params = params.fillna(-1)

if "Охрана:" not in df:
    df = pd.concat([df, params], axis=1)
    
    obj_feats = params.select_dtypes(object).columns

    for feat in obj_feats:
        df["{}_cat".format(feat)] = df[feat].factorize()[0]

cat_feats = [x for x in df.columns if "_cat" in x]

cat_feats

['Охрана:_cat',
 'Тип здания:_cat',
 'Тип объекта:_cat',
 'Количество корпусов:_cat',
 'Тип объявления:_cat',
 'Застройщик:_cat',
 'Общая площадь:_cat',
 'Дата публикации:_cat',
 'Количество комнат:_cat',
 'Парковка:_cat',
 'Дата  обновления:_cat',
 'Количество этажей:_cat',
 'Сдача:_cat',
 'Комиссия агенту:_cat',
 'Высота потолков:_cat',
 'Этаж:_cat',
 'Этап строительства:_cat',
 'Новостройка:_cat',
 'Количество квартир:_cat',
 'Класс жилья:_cat',
 'Адрес:_cat',
 'Лифт:_cat',
 'Вид из окна:_cat',
 'Возможна ипотека:_cat',
 'Год постройки:_cat',
 'Этажность:_cat',
 'Мусоропровод:_cat',
 'Ремонт:_cat',
 'Площадь кухни:_cat',
 'Жилая комната:_cat',
 'Тип санузла:_cat',
 'Тип балкона:_cat',
 'Мебель на кухне:_cat',
 'Холодильник:_cat',
 'Интернет:_cat',
 'Мебель:_cat',
 'Телефон:_cat',
 'Свободная планировка:_cat',
 'Управляющая компания:_cat',
 'Количество подъездов:_cat',
 'Тип дома:_cat',
 'Тип комнат:_cat',
 'Покрытие пола:_cat',
 'Серия:_cat',
 'Тип перекрытия:_cat',
 'Возможен торг:

In [13]:
# powierzchnia mieszkania
df["area"] = df["Общая площадь:"].map(parse_area)

In [14]:
# powierzchnia kuchni
df["kitchen_area"] = df["Площадь кухни:"].map(parse_area)

In [15]:
geo_block = (
df["geo_block"]
    .map(lambda x: x[:int(len(x)/2) ])
    .map(lambda x: {"geo_block_{}".format(idx):val for idx,val in enumerate(x) })
    .apply(pd.Series)
)

In [16]:
for feat in geo_block.columns:
    df["{}_cat".format(feat)] = geo_block[feat].factorize()[0]

In [17]:
geo_cat_feats = [x for x in df.columns if "geo_block" in x and "_cat" in x]

In [18]:
breadcrumbs = (
    df["breadcrumbs"]
        .map(lambda x: {"breadcrumbs_{}".format(idx):val for idx,val in enumerate(x) })
        .apply(pd.Series)
)

In [19]:
for feat in breadcrumbs.columns:
    df["{}_cat".format(feat)] = breadcrumbs[feat].factorize()[0]
df

,geo_block,breadcrumbs,price,owner,params,date,user_block,id,Охрана:,Тип здания:,...,geo_block_4_cat,geo_block_5_cat,breadcrumbs_0_cat,breadcrumbs_1_cat,breadcrumbs_2_cat,breadcrumbs_3_cat,breadcrumbs_4_cat,breadcrumbs_5_cat,breadcrumbs_6_cat,breadcrumbs_7_cat
0,"[г. Москва, Лианозово, г. Москва, Лианозово]","[Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]",5.402084,[],"{'Охрана:': 'закрытая территория', 'Тип здания...","[22 апреля, 12, (+1 за сегодня), Обновлено 18 ...","[<div class=""block-user__name"">НДВ Недвижимост...",34995,закрытая территория,Монолитное,...,-1,-1,0,0,0,0,-1,-1,-1,-1
1,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",6.827990,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 2, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",15030,-1,Монолитное,...,-1,-1,0,1,1,1,-1,-1,-1,-1
2,"[г. Москва, ул Лобачевского, г. Москва, ул Лоб...","[Москва, МЦК Кутузовская, ул. Лобачевского]",8.680000,"[ЖК ""Крылья"". Доп. скидка 5%!]","{'Лифт:': 'да', 'Охрана:': 'закрытая территори...","[20 марта, 12, (+2 за сегодня), Обновлено 18 мая]",[],37811,закрытая территория,Монолитное,...,-1,-1,0,2,2,-1,-1,-1,-1,-1
3,"[г. Москва, Ховрино, ул Дыбенко, г. Москва, Хо...","[Москва, Ховрино, м. Ховрино, МЦК Коптево, ул....",8.624435,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[29 апреля, 3, (+1 за сегодня), Обновлено 13 мая]","[<div class=""block-user__name"">MR-Group</div>,...",31685,-1,Монолитное,...,-1,-1,0,1,1,1,0,-1,-1,-1
4,"[г. Москва, Даниловский, г. Москва, Даниловский]","[Москва, Даниловский, м. Автозаводская, МЦК Ав...",29.500000,[],"{'Общая площадь:': '141.1 м²', 'Дата публикаци...","[15 декабря 2018, 30, (+1 за сегодня), Обновле...","[<div class=""block-user__name"">Наталья Вдовиче...",56215,-1,Монолитное,...,-1,-1,0,3,3,2,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11443,"[г. Москва, г. Москва]","[Москва, МЦК Кутузовская]",NaN,[],"{'Лифт:': 'да', 'Охрана:': 'предусмотрена', 'Т...","[17 мая, 3, (+2 за сегодня)]","[<div class=""block-user__name"">MR Group</div>,...",47294,предусмотрена,Монолитное,...,-1,-1,0,2,-1,-1,-1,-1,-1,-1
11444,"[г. Москва, Хорошёво-Мнёвники, г. Москва, Хоро...","[Москва, Хорошёво-Мнёвники, м. Октябрьское пол...",NaN,[],"{'Лифт:': 'да', 'Охрана:': 'предусмотрена', 'Т...","[16 мая, 1, (+1 за сегодня)]","[<div class=""block-user__name"">Офис продаж ЖК ...",14091,предусмотрена,Монолитно-кирпичное,...,-1,-1,0,22,22,18,-1,-1,-1,-1
11445,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",NaN,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 1, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",13059,-1,Монолитное,...,-1,-1,0,1,1,1,-1,-1,-1,-1
11446,"[Новая Москва, п. Внуковское, г. Москва, Новая...","[Москва, Новая Москва, п. Внуковское, МЦК Куту...",NaN,[Переделкино Ближнее],"{'Лифт:': 'да', 'Охрана:': 'консьерж', 'Тип зд...","[вчера в 10:14, 2, (+2 за сегодня)]",[],10144,консьерж,Монолитное,...,-1,-1,0,5,21,9,-1,-1,-1,-1


In [20]:
breadcrumbs_cat_feats = [x for x in df.columns if "breadcrumbs" in x and "_cat" in x]
breadcrumbs_cat_feats

['breadcrumbs_0_cat',
 'breadcrumbs_1_cat',
 'breadcrumbs_2_cat',
 'breadcrumbs_3_cat',
 'breadcrumbs_4_cat',
 'breadcrumbs_5_cat',
 'breadcrumbs_6_cat',
 'breadcrumbs_7_cat']

In [21]:
metro_station = (
    df["breadcrumbs"]
        .map(lambda x: get_metro_station(x))
        .apply(pd.Series)
)
metro_station.columns = ['metro_station_name']

In [22]:
df["metro_station_cat"] = metro_station.apply(lambda x : pd.factorize(x)[0])

df

,geo_block,breadcrumbs,price,owner,params,date,user_block,id,Охрана:,Тип здания:,...,geo_block_5_cat,breadcrumbs_0_cat,breadcrumbs_1_cat,breadcrumbs_2_cat,breadcrumbs_3_cat,breadcrumbs_4_cat,breadcrumbs_5_cat,breadcrumbs_6_cat,breadcrumbs_7_cat,metro_station_cat
0,"[г. Москва, Лианозово, г. Москва, Лианозово]","[Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]",5.402084,[],"{'Охрана:': 'закрытая территория', 'Тип здания...","[22 апреля, 12, (+1 за сегодня), Обновлено 18 ...","[<div class=""block-user__name"">НДВ Недвижимост...",34995,закрытая территория,Монолитное,...,-1,0,0,0,0,-1,-1,-1,-1,0
1,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",6.827990,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 2, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",15030,-1,Монолитное,...,-1,0,1,1,1,-1,-1,-1,-1,1
2,"[г. Москва, ул Лобачевского, г. Москва, ул Лоб...","[Москва, МЦК Кутузовская, ул. Лобачевского]",8.680000,"[ЖК ""Крылья"". Доп. скидка 5%!]","{'Лифт:': 'да', 'Охрана:': 'закрытая территори...","[20 марта, 12, (+2 за сегодня), Обновлено 18 мая]",[],37811,закрытая территория,Монолитное,...,-1,0,2,2,-1,-1,-1,-1,-1,2
3,"[г. Москва, Ховрино, ул Дыбенко, г. Москва, Хо...","[Москва, Ховрино, м. Ховрино, МЦК Коптево, ул....",8.624435,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[29 апреля, 3, (+1 за сегодня), Обновлено 13 мая]","[<div class=""block-user__name"">MR-Group</div>,...",31685,-1,Монолитное,...,-1,0,1,1,1,0,-1,-1,-1,1
4,"[г. Москва, Даниловский, г. Москва, Даниловский]","[Москва, Даниловский, м. Автозаводская, МЦК Ав...",29.500000,[],"{'Общая площадь:': '141.1 м²', 'Дата публикаци...","[15 декабря 2018, 30, (+1 за сегодня), Обновле...","[<div class=""block-user__name"">Наталья Вдовиче...",56215,-1,Монолитное,...,-1,0,3,3,2,-1,-1,-1,-1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11443,"[г. Москва, г. Москва]","[Москва, МЦК Кутузовская]",NaN,[],"{'Лифт:': 'да', 'Охрана:': 'предусмотрена', 'Т...","[17 мая, 3, (+2 за сегодня)]","[<div class=""block-user__name"">MR Group</div>,...",47294,предусмотрена,Монолитное,...,-1,0,2,-1,-1,-1,-1,-1,-1,2
11444,"[г. Москва, Хорошёво-Мнёвники, г. Москва, Хоро...","[Москва, Хорошёво-Мнёвники, м. Октябрьское пол...",NaN,[],"{'Лифт:': 'да', 'Охрана:': 'предусмотрена', 'Т...","[16 мая, 1, (+1 за сегодня)]","[<div class=""block-user__name"">Офис продаж ЖК ...",14091,предусмотрена,Монолитно-кирпичное,...,-1,0,22,22,18,-1,-1,-1,-1,19
11445,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",NaN,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 1, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",13059,-1,Монолитное,...,-1,0,1,1,1,-1,-1,-1,-1,1
11446,"[Новая Москва, п. Внуковское, г. Москва, Новая...","[Москва, Новая Москва, п. Внуковское, МЦК Куту...",NaN,[Переделкино Ближнее],"{'Лифт:': 'да', 'Охрана:': 'консьерж', 'Тип зд...","[вчера в 10:14, 2, (+2 за сегодня)]",[],10144,консьерж,Монолитное,...,-1,0,5,21,9,-1,-1,-1,-1,2


# Model DecisionTreeRegressor

In [31]:
feats = ["area", "kitchen_area", "metro_station_cat"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats 

model = DecisionTreeRegressor(max_depth=20)
check_log_model(df, feats, DecisionTreeRegressor(max_depth=20))

(2.621609542095153, 0.3085265528676165)

# Model XGBRegressor

In [33]:

md = 20
ne = 700
lr = 0.15

In [34]:
feats = ["area", "kitchen_area", "metro_station_cat"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats 
      
check_log_model(df, feats, xgb.XGBRegressor(max_depth=md, n_estimators=ne, learning_rate=lr, random_state=0))

(2.780384659375134, 0.3150035304284564)

# Kaggle submit

In [35]:
feats = ["area", "kitchen_area", "metro_station_cat"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats

df_train = df[ ~df["price"].isnull() ].copy()
df_test = df[ df["price"].isnull() ].copy()

X_train = df_train[feats]
y_train = df_train["price"]
y_log_train = np.log(y_train)

X_test = df_test[feats]

model = xgb.XGBRegressor(max_depth=8, n_estimators=700, learning_rate=0.1, random_state=0)
model.fit(X_train, y_log_train)
y_log_pred = model.predict(X_test)
y_pred = np.exp(y_log_pred)


df_test["price"] = y_pred
df_test[ ["id", "price"] ].to_csv("./xgb_location_log_area_v2.csv", index=False)